In [438]:
import pandas as pd
import requests
import datapackage
import pycountry_convert as pc

In [439]:
df = pd.read_csv('pyladies_chapter_leaders_survey.csv') #, index_col=0)
df.shape

(66, 7)

In [440]:
df.columns

Index(['Timestamp', 'Which PyLadies chapter do you belong to?',
       'When was your last event?',
       'What are the challenges your chapter has faced?',
       'How can the PyLadies leadership team help you run your chapter better?',
       'email', 'Email Address'],
      dtype='object')

In [441]:
df.rename(columns={
    'Timestamp': 'timestampe', 
    'Which PyLadies chapter do you belong to?': 'name',
    'When was your last event?': 'last_event',
    'What are the challenges your chapter has faced?': 'challenges',
    'How can the PyLadies leadership team help you run your chapter better?': 'improvement',
    'email': 'intended_individual_email', 
    'Email Address':'intended_chapter_email'
}, inplace=True)

In [442]:
df.columns

Index(['timestampe', 'name', 'last_event', 'challenges', 'improvement',
       'intended_individual_email', 'intended_chapter_email'],
      dtype='object')

In [443]:
num_responses = df.name.count()
num_chapters = df.name.nunique()

In [444]:
df.name

0                                       Brisbane
1                               PyLadies Nigeria
2                               PyLadies Hamburg
3                                      Wellesley
4                                PyLadies London
5                                         London
6                                          Miami
7                                       Brasilia
8                           Pyladies Addis Ababa
9                                  San Francisco
10                               Pyladies London
11                                     Vancouver
12                              Oakland PyLadies
13                               Pyladies La Paz
14                                       Seattle
15                                     Fortaleza
16                                     Melbourne
17                                       Effurun
18                                       Chicago
19                                Chennai, India
20                  

In [445]:
print(f'Total number of responses: {num_responses}\nTotal number of chapters: {num_chapters}')

Total number of responses: 66
Total number of chapters: 65


In [446]:
def standardize_chapter_name(text):
    if ',' in text:
        text = text.split(',')
        text = ' ' .join(word.capitalize() for word in text[0:-1])
    if 'pyladies' in text.lower().split(' '):
        text = text.lower().split(' ')
        text.remove('pyladies')
        if len(text) == 1:
            return text[0].capitalize()
        return ' ' .join(word.capitalize() for word in text)
    
    return text.lower().title()


In [447]:
df.name = df.name.apply(standardize_chapter_name)
print(df.shape)
df.name

(66, 7)


0                                       Brisbane
1                                        Nigeria
2                                        Hamburg
3                                      Wellesley
4                                         London
5                                         London
6                                          Miami
7                                       Brasilia
8                                    Addis Ababa
9                                  San Francisco
10                                        London
11                                     Vancouver
12                                       Oakland
13                                        La Paz
14                                       Seattle
15                                     Fortaleza
16                                     Melbourne
17                                       Effurun
18                                       Chicago
19                                       Chennai
20                  

In [448]:
package = Package('https://datahub.io/core/world-cities/datapackage.json')

data_url = 'https://datahub.io/core/world-cities/datapackage.json'

package = datapackage.Package(data_url)

worldcities_df = None

resources = package.resources
for resource in resources:
    if resource.tabular:
        worldcities_df = pd.read_csv(resource.descriptor['path'])

worldcities_df.head()

,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm al Qaywayn,United Arab Emirates,Umm al Qaywayn,290594
3,Ras al-Khaimah,United Arab Emirates,Raʼs al Khaymah,291074
4,Khawr Fakkān,United Arab Emirates,Ash Shāriqah,291696


In [449]:
with_countries_df = pd.merge(df,
                             worldcities_df[['name', 'country', 'subcountry']],
                             on='name',
                             how='left')

print(with_countries_df.shape)
with_countries_df.iloc[0]

(100, 9)


timestampe                                 8/14/2019 20:37:25
name                                                 Brisbane
last_event                                           8/9/2019
challenges                                    Engaging people
improvement                  Advertising the PyLadies groups 
intended_individual_email                                 NaN
intended_chapter_email                  brisbane@pyladies.com
country                                             Australia
subcountry                                         Queensland
Name: 0, dtype: object

In [450]:
with_countries_df['duplicated'] = with_countries_df['timestampe'].duplicated()

In [451]:
# Get list of duplicated emails, and find the rows with these duplicated emails
duplicated_names = with_countries_df[with_countries_df['duplicated'] == True].intended_chapter_email.unique()
duplicated_rows = with_countries_df[with_countries_df.intended_chapter_email.isin(duplicated_names)]

# Print this duplicated set of rows
print(len(duplicated_rows))
print(duplicated_rows[['name', 'country']])  # 'intended_chapter_email'

52
             name         country
4          London          Canada
5          London  United Kingdom
6          London          Canada
7          London  United Kingdom
11  San Francisco       Argentina
12  San Francisco      Costa Rica
13  San Francisco     Philippines
14  San Francisco     Philippines
15  San Francisco     El Salvador
16  San Francisco   United States
17         London          Canada
18         London  United Kingdom
19      Vancouver          Canada
20      Vancouver   United States
22         La Paz       Argentina
23         La Paz         Bolivia
24         La Paz        Honduras
25         La Paz          Mexico
26         La Paz     Philippines
27         La Paz         Uruguay
30      Melbourne       Australia
31      Melbourne   United States
35         Newark   United States
36         Newark   United States
37         Newark   United States
38         Newark   United States
44       Campinas          Brazil
45       Campinas          Brazil
47         

# Validate which are incorrectly paired with a country

* London, UK: 
   - [MeetUp Organizers](https://www.meetup.com/PyLadiesLondon/members/?op=leaders)
      - Correct rows with IDs: 5, 7, 18
      - Drop IDs: 4, 6, 17
* San Francisco, USA:
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in USA 
      - Correct rows with IDs: 16
      - Drop IDs: 11, 12, 13, 14, 15
* Vancouver, Canada:
   - [MeetUp Organizers](https://www.meetup.com/PyLadies-Vancouver/members/?op=leaders)
     - Correct rows with IDs: 19, 92
     - Drop IDs: 20, 93 
* La Paz, Bolivia:
  - [MeetUp Organizers](https://www.meetup.com/La-Paz-PyLadies-Meetup/members/?op=leaders)
     - Correct rows: 23
     - Drop IDS: 22, 24, 25, 26, 27
* Melbourne, Australia
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in Australia 
    - Correct rows: 30
    - Drop IDs: 31
* Newark, Delware
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in USA 
    - Correct rows: 35
    - Drop IDs: 36, 37, 38
* Campinas, Brasil:
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in Brasil 
    - Correct rows: 44
    - Drop IDs: 45 
* Lima, Peru:
  - [PyLadies Lima writeup](https://medium.com/@karen_dax/1er-meetup-de-pyladies-lima-1214988ea711) 
    - Correct rows: 47
    - Drop IDs: 48  
* Santa Cruz, USA:
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in USA 
      - Correct rows with IDs: 61
      - Drop IDs: 56, 57, 58, 59, 60
* Sydney, Australia:
  - [MeetUp Organizers](https://www.meetup.com/en-AU/Sydney-PyLadies/members/?op=leaders)
     - Correct rows: 62
     - Drop IDS: 63
* Amsterdam, Netherleands:
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in Netherlands
    - Correct rows: 66
    - Drop IDs: 67
* Dublin, Ireland
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in Ireland 
    - Correct rows: 71
    - Drop IDs: 72, 73, 74 
* Belem, Brasil
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in Brasil 
    - Correct rows: 86
    - Drop IDs: 87  
* Madrid, Spain
  - [Pyladies Chapters](https://www.pyladies.com/locations/) only has one location in Spain (found on MeetUp); [MeetUp Organizers](https://www.meetup.com/es-ES/PyLadiesMadrid/)
    - Correct rows: 89, 99
    - Drop IDs: 88, 98


correct_ids = [5, 7, 18, 16, 19, 92, 23, 30, 35, 44, 47, 61, 62, 66, 71, 86, 89, 99]
drop_ids = [4, 6, 17, 11, 12, 13, 14, 15, 20, 93, 22, 24, 25, 26, 27, 31, 36, 37, 38, 45, 48, 56, 57, 58, 59, 60, 63, 67, 72, 73, 74, 87, 88, 98]

In [453]:
correct_ids = [5, 7, 18, 16, 19, 92, 23, 30, 35, 44, 47, 61, 62, 66, 71, 86, 89, 99]
drop_ids = [4, 6, 17, 11, 12, 13, 14, 15, 20, 93, 22, 24, 25, 26, 27, 31, 36, 37, 38, 45, 48, 56, 57, 58, 59, 60, 63, 67, 72, 73, 74, 87, 88, 98]

values_equal = with_countries_df.shape[0] - (len(drop_ids)) == df.shape[0]
# print(with_countries_df.shape[0] - (len(drop_ids)), df.shape[0])
# print(set(duplicated_rows.index) - set(drop_ids + correct_ids))

print(f'Are the sizes of the two dataframes equal: {values_equal}')

with_countries_df = with_countries_df.drop(drop_ids)
with_countries_df = with_countries_df.drop(columns=['duplicated'])

# print(with_countries_df.shape[0])

Are the sizes of the two dataframes equal: True


In [454]:
def convert_country_to_contient(country):
    if not isinstance(country, str):
        return 'N/A'
    country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
    return pc.country_alpha2_to_continent_code(country_code)

In [455]:
with_countries_df['continent_code'] = with_countries_df.country.apply(convert_country_to_contient)

In [456]:
na_continent = with_countries_df[with_countries_df.continent_code == 'N/A']
print(f'{len(na_continent)} are unidentified.')
print(na_continent.name)

13 are unidentified.
1                                        Nigeria
9                                       Brasilia
32                                       Effurun
42                                  Northwest Uk
43                                    Central Pa
46                                   Bhubaneswar
51                                         Ghana
52                                   Pyladiesrgv
53                                           Rdu
55                                Rio De Janeiro
81    Rural Federal University Of Rio De Janeiro
83                                       Paraiba
95                                        Brazil
Name: name, dtype: object


In [457]:
with_countries_df.at[1, 'country'] = 'Nigeria'
with_countries_df.at[9, 'country'] = 'Brazil'
with_countries_df.at[32, 'country'] = 'Nigeria'
with_countries_df.at[42, 'country'] = 'United Kingdom'
with_countries_df.at[43, 'country'] = 'United States'
with_countries_df.at[46, 'country'] = 'India'
with_countries_df.at[51, 'country'] = 'Ghana'
with_countries_df.at[52, 'country'] = 'United States'   # https://twitter.com/pyladiesrgv
with_countries_df.at[53, 'country'] = 'United States'   # https://www.meetup.com/pyladies-rdu/
with_countries_df.at[55, 'country'] = 'Brazil'
with_countries_df.at[81, 'country'] = 'Brazil'
with_countries_df.at[83, 'country'] = 'Brazil'
with_countries_df.at[95, 'country'] = 'Brazil'

In [458]:
with_countries_df['continent_code'] = with_countries_df.country.apply(convert_country_to_contient)

In [459]:
na_continent = with_countries_df[with_countries_df.continent_code == 'N/A']
na_continent.shape[0]

0

In [436]:
export_csv = with_countries_df.to_csv('pyladies_chapters_survey_with_country_info.csv', index = None, header=True)

In [469]:
unique_chapters = with_countries_df.name.unique()
print(f'Number of unique chapter responses: {len(unique_chapters)}')
print(f'Here are the chapters: {unique_chapters}')

Number of unique chapter responses: 59
Here are the chapters: ['Brisbane' 'Nigeria' 'Hamburg' 'Wellesley' 'London' 'Miami' 'Brasilia'
 'Addis Ababa' 'San Francisco' 'Vancouver' 'Oakland' 'La Paz' 'Seattle'
 'Fortaleza' 'Melbourne' 'Effurun' 'Chicago' 'Chennai' 'Newark'
 'Mexico City' 'Okinawa' 'Grand Rapids' 'Northwest Uk' 'Central Pa'
 'Campinas' 'Bhubaneswar' 'Lima' 'Arequipa' 'Doboj' 'Ghana' 'Pyladiesrgv'
 'Rdu' 'Berlin' 'Rio De Janeiro' 'Santa Cruz' 'Sydney' 'São Luís'
 'Salt Lake City' 'Amsterdam' 'Goiânia' 'Goa' 'Bangkok' 'Dublin' 'Recife'
 'Houston' 'Paris' 'São Paulo' 'Kampala'
 'Rural Federal University Of Rio De Janeiro' 'Teresina' 'Paraiba'
 'Maceió' 'Belém' 'Madrid' 'Natal' 'São Carlos' 'Manaus' 'Brazil'
 'New York City']


In [471]:
uniquewith_countries_df.country.unique())
print()

22